In [1]:
from dataset import Kitti, get_dataloader
import pandas as pd
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [47]:
from pcdet.utils.box_utils import boxes_to_corners_3d, remove_points_in_boxes3d, remove_points_outside_boxes3d

In [10]:
kitti_data = Kitti(data_root='data/kitti', 
                    split='train')
example = kitti_data.__getitem__(9)
example['pts'].shape
box = example['gt_bboxes_3d'][0]

In [11]:
sample1 = kitti_data.__getitem__(23)
sample2 = kitti_data.__getitem__(2323)

In [110]:
def rotation_3d_in_axis(pts, theta, axis=0):
    if len(pts.shape) == 2:
        pts = np.expand_dims(pts, axis=0)
        
    rot_sin = np.sin(theta)
    rot_cos = np.cos(theta)
    ones = np.ones_like(rot_cos)
    zeros = np.zeros_like(rot_cos)

    if axis == 1:
        rot_mat_T = np.stack([[rot_cos, zeros, -rot_sin], [zeros, ones, zeros],
                              [rot_sin, zeros, rot_cos]])
    elif axis == 2 or axis == -1:
        rot_mat_T = np.stack([[rot_cos, -rot_sin, zeros],
                              [rot_sin, rot_cos, zeros], [zeros, zeros, ones]])
    elif axis == 0:
        rot_mat_T = np.stack([[zeros, rot_cos, -rot_sin],
                              [zeros, rot_sin, rot_cos], [ones, zeros, zeros]])
    else:
        raise ValueError("axis should in range")

    return np.einsum('aij,jka->aik', pts, rot_mat_T)

In [113]:
def swap_octodrons(data_dict, distance_limit=100, p=1.0):

    pts, gt_bboxes_3d, gt_labels = data_dict['pts'], data_dict['gt_bboxes_3d'], data_dict['gt_labels']
    gt_box_corners = boxes_to_corners_3d(gt_bboxes_3d)

    for label in range(3):
        label_idx = list(np.where(gt_labels == label)[0])

        for i in label_idx:
            # only apply swap when x is smaller than the distance limit
            if gt_bboxes_3d[i][0] > distance_limit:
                continue

            # randomly apply swap with a probability of p:
            if np.random.rand(1) > p:
                continue

            # remove i from label idx... only swap it once and it does not swap with it self
            label_idx.remove(i)

            # corner points of the current box
            cur_box_corner_pts = gt_box_corners[i]

            # find non-empty partition idx for both gt
            chosen_octodron_idx = -1 

            # vars for further processing. init as empty
            cur_oct = None
            target_pts = None
            target_gt_idx = -1

            while len(label_idx) > 0:
                target_gt_idx = np.random.choice(label_idx, 1, replace=False)[0]
                target_box_corner_pts = gt_box_corners[target_gt_idx]

                for idx_octodron in range(8):
                    target_oct_centers = (gt_bboxes_3d[target_gt_idx][:3] + target_box_corner_pts[idx_octodron]) / 2
                    target_oct_dims = gt_bboxes_3d[target_gt_idx][3:-1] / 2

                    target_oct = np.hstack((target_oct_centers, target_oct_dims, gt_bboxes_3d[target_gt_idx][-1]))
                    target_pts = remove_points_outside_boxes3d(pts, target_oct[np.newaxis, :])

                    if len(target_pts) == 0:
                        continue
                    
                    # print(gt_bboxes_3d[i][:3])
                    cur_oct_centers = (gt_bboxes_3d[i][:3] + cur_box_corner_pts[idx_octodron]) / 2
                    cur_oct_dims = gt_bboxes_3d[i][3:-1] / 2

                    cur_oct = np.hstack((cur_oct_centers, cur_oct_dims, gt_bboxes_3d[i][-1]))
                    cur_pts = remove_points_outside_boxes3d(pts, cur_oct[np.newaxis, :])

                    if len(cur_pts) > 0:
                        chosen_octodron_idx = idx_octodron
                        break
                
                label_idx.remove(target_gt_idx)
            
            if chosen_octodron_idx == -1:
                continue

            target_box_center, target_box_dim, target_box_heading = gt_bboxes_3d[target_gt_idx][:3], gt_bboxes_3d[target_gt_idx][3:-1], gt_bboxes_3d[target_gt_idx][6:7]
            # transform target octodron points
            target_pts[:, :3] -= target_box_center
            target_pts[:, :3] = rotation_3d_in_axis(target_pts[:, :3], -target_box_heading, axis=2)
            target_pts[:, :3] /= target_box_dim

            # restore target points to the current box
            cur_box_center, cur_box_dim, cur_box_heading = gt_bboxes_3d[i][:3], gt_bboxes_3d[i][3:-1], gt_bboxes_3d[i][6:7]
            target_pts[:, :3] *= cur_box_dim
            target_pts[:, :3] = rotation_3d_in_axis(target_pts[:, :3], cur_box_heading, axis=2)
            target_pts[:, :3] += cur_box_center

            # swap
            pts = remove_points_in_boxes3d(pts, cur_oct[np.newaxis, :])
            pts = np.vstack((pts, target_pts))

    data_dict['pts'] = pts

    return data_dict

In [114]:
pts, gt_bboxes_3d, gt_labels = sample1['pts'], sample1['gt_bboxes_3d'], sample1['gt_labels']
gt_box_corners = boxes_to_corners_3d(gt_bboxes_3d)

target_gt_idx = 17
idx_octodron = 3

target_box_corner_pts = gt_box_corners[target_gt_idx]
(gt_bboxes_3d[target_gt_idx][:3] + target_box_corner_pts[idx_octodron]) / 2

array([ 3.4467587,  6.026842 , -1.5149769], dtype=float32)

In [129]:
print(sample1['pts'].shape)
swap_octodrons(sample1)['pts'].shape

(38288, 4)


(38287, 4)

In [44]:
gt_bboxes_3d = sample1['gt_bboxes_3d']
pts = sample1['pts']
gt_box_corners = boxes_to_corners_3d(gt_bboxes_3d)
target_gt_idx = 23
target_box_corner_pts = gt_box_corners[target_gt_idx]
target_oct_centers = (gt_bboxes_3d[target_gt_idx][:3] + target_box_corner_pts[7]) / 2
target_oct_dims = gt_bboxes_3d[target_gt_idx][3:-1] / 2
target_oct = np.hstack((target_oct_centers, target_oct_dims, gt_bboxes_3d[target_gt_idx][-1]))
oct_pts = remove_points_outside_boxes3d(pts, target_oct[np.newaxis, :])

In [52]:
print(pts.shape)
print(oct_pts.shape)
np.vstack((oct_pts, pts)).shape

(38259, 4)
(5, 4)


(38264, 4)

In [23]:
pts.shape

(38259, 4)

In [25]:
target_box_corner_pts

array([[ -1.3623054, -33.154373 ,  -2.4427369],
       [  2.8396785, -32.234016 ,  -2.4427369],
       [  3.1669166, -33.728058 ,  -2.4427369],
       [ -1.0350673, -34.648415 ,  -2.4427369],
       [ -1.3623054, -33.154373 ,  -1.1331398],
       [  2.8396785, -32.234016 ,  -1.1331398],
       [  3.1669166, -33.728058 ,  -1.1331398],
       [ -1.0350673, -34.648415 ,  -1.1331398]], dtype=float32)

In [14]:
target_oct_centers

array([ -0.22999987, -33.297794  ,  -2.1153376 ], dtype=float32)